> Enterprise Web C#

# Chapter 3 - Big O(h) Oh collections

## Introduction

LINQ stands for **L**anguage **IN**tegrated **Q**uery. It's a query language integrated in C# providing an SQL-like interface for a variety of **data sources**. Because it's completely integrated, it features IntelliSense and compile time checking amongst other features provided by the compiler.

### Data source

LINQ is defined for a number of data sources, but what is a data source? A data source is the data that is being queried and which has a LINQ provider. There are a number of LINQ providers:

- LINQ to Objects
  - querying in-memory data sources like strings, arrays, collections...
  - mainly the focus in this chapter
- LINQ to SQL
  - querying data in a relational database from C#
- LINQ to Entities
  - similar to LINQ to SQL but using [Entity Framework](https://docs.microsoft.com/en-us/ef/)
  - will be covered in a later chapter

Once you master the LINQ syntax, you can query any data source which provides a LINQ interface:
- LINQ to
  - Google, Twitter, eBay, Amazon, Fliqr...
  - XML, JSON...
  - MySQL, Oracle...
  - Excel, Word...
  - JavaScript...
  - ...

### LINQ syntax

There are two types of LINQ syntax: **query syntax** or **method syntax**.

### Query syntax

```cs
var query = from c in customerList
            where c.CustomerId == customerId
            select c;
```

The query syntax is a declarative and builtin syntax. The compiler will translate the query syntax to method syntax at compile time. Fewer LINQ operators are available when using this syntax.

### Method syntax

```cs
var query = customerList.Where(c => c.CustomerId == customerId);
```

The method syntax uses methods to query a data source. It's a part of the .NET framework: the `System.Linq` namespace in the `System.Core` assembly.

## LINQ to Objects

### Extension methods

Extension methods are methods added to an existing class to extend this class' functionality. This functionality can be added without creating a subclass, modifying the original class or even recompiling the original class. They don't differ from instance methods (methods declared in the class itself).

An extension method is declared static within a non generic static class. The first parameter must be preceded by the keyword `this`. The type of this first parameter is the type that is being extended. All parameters other parameters must be given when using the method.

```cs
public static class IntExtension
{
  public static bool IsEven(this int i)
  {
    return i % 2 == 0;
  }
}
```

This extension method has no parameters and can be called on any int: `10.IsEven()`.

```cs
public static class IntExtension
{
  public static bool Add(this int i, int number)
  {
    return i + number;
  }
}
```

This extension method has one parameters and can be called on any int: `10.Add(2)`.

An extension method has no access to private member of the class for which it's declared. The extension method cannot override an instance method (same name and arguments), the compiler will always choose the instance method.

Extension methods are best declared in another namespace named `Extensions`. This way you must explicitly write `using Extensions;` in order to use the extensions.

This feature cannot be tested in .NET Interactive.... For demos and exercises, please use Visual Studio. An example can be found [here](https://github.com/HOGENT-Web/csharp-ch-4-example-1).
        

